<a href="https://colab.research.google.com/github/Letch49/ML-vvsu-2025-2026/blob/master/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# МЛ Практика 5.1

## Цель занятия

Изучить автоматизированные методы извлечения влиящих характеристик, познакомится с ColumnTransformer, Pipeline, CrossValidation.

Финализация работы с табличными данными

In [13]:
import pandas as pd
from pprint import pprint

In [5]:
df = pd.read_csv('/content/wine.csv')

In [8]:
df.head(n=2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red


In [7]:
df.quality = df.quality.astype('category')
df['type'] = df['type'].astype('category')

### Объявим для двух решаемых задач два разных признака

In [9]:
y_regression = df['alcohol']
X_regression = df.drop(columns=['alcohol'])

## Часть 1 - автоматизированный подбор признаков

Для отбора признаков, существуют `3 типа методов`:

- Фильтрационные
- Оберточные
- Встроенные

## Фильтрационные методы

`Filter method` - методы, основанные на сравнении пар `один признак` -> `y`. Рассматривается базовое сравнение признаков.

`y` - регрессия

**f_regression**:
- F-тест на значимость линейной зависимости между Xᵢ и y.
- Чем больше F-статистика, тем сильнее линейная связь (для категориальных признаков предварительно сделать OneHot).

**r_regression**:
- Коэф. корреляции пирсона (важная особенность, r в данном случае берется по модулю)
- |r| ≈ 1 → сильная линейная связь, |r| ≈ 0 → слабая.

**mutual_info_regression**:
- Взаимная информация: насколько знание Xᵢ уменьшает неопределённость y.
Ловит нелинейные зависимости, может работать с дискретными X.

----------

`y` - категориальная

**f_classif**:
- ANOVA F-тест: сравнивает среднее значения признака между классами.
Подходит для числовых X, y – классы.

**chi2**
- χ²-критерий независимости: для неотрицательных признаков (частоты, 0/1 после OHE).

**mutual_info_classif**:
- Взаимная информация между признаком и классом.
Работает и с числовыми, и с категориальными признаками (нужно пометить дискретные).


| Тип признака X | Тип целевой y                  | Что используем в sklearn                                                       | Комментарий                             |
| -------------- | ------------------------------ | ------------------------------------------------------------------------------ | --------------------------------------- |
| числовой       | числовая (регрессия)           | `f_regression`, `r_regression`, `mutual_info_regression`                       | Линейная (f, r) и нелинейная (MI) связь |
| категориальный | числовая                       | `f_regression` (после OneHot), `mutual_info_regression(discrete_features=...)` | Категории → набор бинарных фич          |
| числовой       | категориальная (классификация) | `f_classif`, `mutual_info_classif`                                             | Разница средних между классами, MI      |
| категориальный | категориальная                 | `chi2` (после OneHot), `mutual_info_classif(discrete_features=...)`            | Таблица сопряжённости, MI               |


Все эти методы используются через специальные обретки

- SelectKBest - выбор n лучших фич

- SelectPercentile - выбор фич по перцентилю

- GenericUnivariateSelect - универсальный метод подбора, тип выбора опреедляется по параметру mode, и param, где mode - режим подбора, param - коэфициент

In [21]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

### Отбор признаков для задачи регрессии (Фильтрационные методы)

In [19]:
numeric_features = X_regression.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_regression.select_dtypes(include=['category']).columns

In [22]:
# делаем преобразование через Pipeline и Transformer. Об этом чуть позже
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [24]:
prep_pipe = Pipeline(steps=[
    ('preprocess', preprocessor)
])

In [31]:
X_reg_prepared = prep_pipe.fit_transform(X_regression)
X_reg_prepared

array([[ 0.14247327,  2.18883292, -2.19283252, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.45103572,  3.28223494, -2.19283252, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.45103572,  2.55330026, -1.91755268, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.55179227, -0.6054167 , -0.88525328, ...,  0.        ,
         0.        ,  1.        ],
       [-1.32319841, -0.30169391, -0.12823371, ...,  0.        ,
         0.        ,  1.        ],
       [-0.93749534, -0.78765037,  0.42232597, ...,  0.        ,
         0.        ,  1.        ]])

In [32]:
feature_names_reg = prep_pipe.named_steps['preprocess'].get_feature_names_out()
feature_names_reg

array(['num__fixed acidity', 'num__volatile acidity', 'num__citric acid',
       'num__residual sugar', 'num__chlorides',
       'num__free sulfur dioxide', 'num__total sulfur dioxide',
       'num__density', 'num__pH', 'num__sulphates', 'cat__quality_3',
       'cat__quality_4', 'cat__quality_5', 'cat__quality_6',
       'cat__quality_7', 'cat__quality_8', 'cat__quality_9',
       'cat__type_red', 'cat__type_white'], dtype=object)

In [33]:
pd.DataFrame(data=X_reg_prepared, columns=feature_names_reg)

,num__fixed acidity,num__volatile acidity,num__citric acid,num__residual sugar,num__chlorides,num__free sulfur dioxide,num__total sulfur dioxide,num__density,num__pH,num__sulphates,cat__quality_3,cat__quality_4,cat__quality_5,cat__quality_6,cat__quality_7,cat__quality_8,cat__quality_9,cat__type_red,cat__type_white
0,0.142473,2.188833,-2.192833,-0.744778,0.569958,-1.100140,-1.446359,1.034993,1.813090,0.193097,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.451036,3.282235,-2.192833,-0.597640,1.197975,-0.311320,-0.862469,0.701486,-0.115073,0.999579,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.451036,2.553300,-1.917553,-0.660699,1.026697,-0.874763,-1.092486,0.768188,0.258120,0.797958,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3.073817,-0.362438,1.661085,-0.744778,0.541412,-0.762074,-0.986324,1.101694,-0.363868,0.327510,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.142473,2.188833,-2.192833,-0.744778,0.569958,-1.100140,-1.446359,1.034993,1.813090,0.193097,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,-0.783214,-0.787650,-0.197054,-0.807837,-0.486252,-0.367664,-0.420128,-1.186161,0.320319,-0.210144,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
6493,-0.474652,-0.119460,0.284686,0.537425,-0.257883,1.491697,0.924588,0.067824,-0.426067,-0.478971,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6494,-0.551792,-0.605417,-0.885253,-0.891916,-0.429160,-0.029599,-0.083949,-0.719251,-1.421248,-0.478971,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
6495,-1.323198,-0.301694,-0.128234,-0.912936,-0.971538,-0.593041,-0.101642,-2.003251,0.755710,-1.016626,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


#### SelectKBest + f_regression

In [34]:
from sklearn.feature_selection import SelectKBest, f_regression

k = 8  # сколько признаков оставить

selector_f = SelectKBest(score_func=f_regression, k=k)
X_f_selected = selector_f.fit_transform(X_reg_prepared, y_regression)

mask_f = selector_f.get_support()
selected_features_f = feature_names_reg[mask_f]
scores_f = selector_f.scores_[mask_f]

print("Признаки, выбранные по f_regression:")
for name, score in zip(selected_features_f, scores_f):
    print(f"{name:40s}  F-score = {score:.3f}")


Признаки, выбранные по f_regression:
num__residual sugar                       F-score = 963.479
num__chlorides                            F-score = 459.003
num__free sulfur dioxide                  F-score = 217.081
num__total sulfur dioxide                 F-score = 493.512
num__density                              F-score = 5797.273
cat__quality_5                            F-score = 1123.780
cat__quality_7                            F-score = 818.837
cat__quality_8                            F-score = 203.123


#### SelectKBest + r_regression

In [39]:
from sklearn.feature_selection import r_regression

k = 5 # сколько фич оставить

selector_r = SelectKBest(score_func=r_regression, k=k)
X_r_selected = selector_r.fit_transform(X_reg_prepared, y_regression)

mask_r = selector_r.get_support()
selected_features_r = feature_names_reg[mask_r]
scores_r = selector_r.scores_[mask_r]

print("Признаки, выбранные по r_regression (модуль корреляции):")
for name, score in zip(selected_features_r, scores_r):
    print(f"{name:40s}  |r| = {score:.3f}")


Признаки, выбранные по r_regression (модуль корреляции):
num__pH                                   |r| = 0.121
cat__quality_6                            |r| = 0.071
cat__quality_7                            |r| = 0.335
cat__quality_8                            |r| = 0.174
cat__quality_9                            |r| = 0.039


#### SelectKBest + mutual_info_regression

In [40]:
from sklearn.feature_selection import mutual_info_regression

selector_mi = SelectKBest(score_func=mutual_info_regression, k=k)
X_mi_selected = selector_mi.fit_transform(X_reg_prepared, y_regression)

mask_mi = selector_mi.get_support()
selected_features_mi = feature_names_reg[mask_mi]
scores_mi = selector_mi.scores_[mask_mi]

print("Признаки, выбранные по mutual_info_regression:")
for name, score in zip(selected_features_mi, scores_mi):
    print(f"{name:40s}  MI = {score:.3f}")


Признаки, выбранные по mutual_info_regression:
num__residual sugar                       MI = 0.331
num__chlorides                            MI = 0.257
num__free sulfur dioxide                  MI = 0.155
num__total sulfur dioxide                 MI = 0.302
num__density                              MI = 0.652


### Отбор признаков для задачи классификации (Фильтрационные методы)

In [41]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [45]:
# целевая и признаки
y_classification = df['quality']

X_classification = df.drop(columns=['quality'])

# разделяем типы признаков
numeric_features_clf = X_classification.select_dtypes(include=['float64', 'int64']).columns
categorical_features_clf = X_classification.select_dtypes(include=['category']).columns

numeric_transformer_clf = Pipeline(steps=[
    ('scaler', MinMaxScaler())   # [0,1]
])

categorical_transformer_clf = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_clf = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_clf, numeric_features_clf),
        ('cat', categorical_transformer_clf, categorical_features_clf)
    ]
)

prep_pipe_clf = Pipeline(steps=[
    ('preprocess', preprocessor_clf)
])

# получаем полностью числовую матрицу признаков
X_clf_prepared = prep_pipe_clf.fit_transform(X_classification)
feature_names_clf = prep_pipe_clf.named_steps['preprocess'].get_feature_names_out()

In [47]:
pd.DataFrame(data=X_clf_prepared, columns=feature_names_clf)

,num__fixed acidity,num__volatile acidity,num__citric acid,num__residual sugar,num__chlorides,num__free sulfur dioxide,num__total sulfur dioxide,num__density,num__pH,num__sulphates,num__alcohol,cat__type_red,cat__type_white
0,0.297521,0.413333,0.000000,0.019939,0.111296,0.034722,0.064516,0.206092,0.612403,0.191011,0.202899,1.0,0.0
1,0.330579,0.533333,0.000000,0.030675,0.147841,0.083333,0.140553,0.186813,0.372093,0.258427,0.260870,1.0,0.0
2,0.330579,0.453333,0.024096,0.026074,0.137874,0.048611,0.110599,0.190669,0.418605,0.241573,0.260870,1.0,0.0
3,0.611570,0.133333,0.337349,0.019939,0.109635,0.055556,0.124424,0.209948,0.341085,0.202247,0.260870,1.0,0.0
4,0.297521,0.413333,0.000000,0.019939,0.111296,0.034722,0.064516,0.206092,0.612403,0.191011,0.202899,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,0.198347,0.086667,0.174699,0.015337,0.049834,0.079861,0.198157,0.077694,0.426357,0.157303,0.463768,0.0,1.0
6493,0.231405,0.160000,0.216867,0.113497,0.063123,0.194444,0.373272,0.150183,0.333333,0.134831,0.231884,0.0,1.0
6494,0.223140,0.106667,0.114458,0.009202,0.053156,0.100694,0.241935,0.104685,0.209302,0.134831,0.202899,0.0,1.0
6495,0.140496,0.140000,0.180723,0.007669,0.021595,0.065972,0.239631,0.030461,0.480620,0.089888,0.695652,0.0,1.0


#### SelectPercentile + f_classif (смысл в том, чтобы выбирать признаки по % признаков от F статстики)

ANOVA (различие средних между классами).

In [49]:
from sklearn.feature_selection import SelectPercentile, f_classif

In [52]:
percentile = 30  # возьмём, например, 30% лучших признаков

selector_f = SelectPercentile(score_func=f_classif, percentile=percentile)
X_f_selected = selector_f.fit_transform(X_clf_prepared, y_classification)

mask_f = selector_f.get_support()
selected_features_f = feature_names_clf[mask_f]
scores_f = selector_f.scores_[mask_f]

print(f"Признаки, выбранные SelectPercentile + f_classif (top {percentile}%):")
for name, score in zip(selected_features_f, scores_f):
    print(f"{name:40s}  F-score = {score:.3f}")

Признаки, выбранные SelectPercentile + f_classif (top 30%):
num__volatile acidity                     F-score = 96.674
num__chlorides                            F-score = 50.850
num__density                              F-score = 136.951
num__alcohol                              F-score = 320.593


#### SelectPercentile + chi2

In [53]:
from sklearn.feature_selection import chi2, SelectPercentile

selector_chi2 = SelectPercentile(score_func=chi2, percentile=percentile)
X_chi2_selected = selector_chi2.fit_transform(X_clf_prepared, y_classification)

mask_chi2 = selector_chi2.get_support()
selected_features_chi2 = feature_names_clf[mask_chi2]
scores_chi2 = selector_chi2.scores_[mask_chi2]

print(f"Признаки, выбранные SelectPercentile + chi2 (top {percentile}%):")
for name, score in zip(selected_features_chi2, scores_chi2):
    print(f"{name:40s}  chi2 = {score:.3f}")


Признаки, выбранные SelectPercentile + chi2 (top 30%):
num__volatile acidity                     chi2 = 37.088
num__alcohol                              chi2 = 122.880
cat__type_red                             chi2 = 87.860
cat__type_white                           chi2 = 28.683


#### GenericUnivariateSelect + mutual_info_classif

In [57]:
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif

percentile = 50

selector_mi = GenericUnivariateSelect(
    score_func=mutual_info_classif,
    mode='percentile',     # можно поменять на 'fpr', 'fdr', 'fwe', 'k_best'
    param=percentile       # параметр зависит от режима (здесь это процент)
)

X_mi_selected = selector_mi.fit_transform(X_clf_prepared, y_classification)

mask_mi = selector_mi.get_support()
selected_features_mi = feature_names_clf[mask_mi]
scores_mi = selector_mi.scores_[mask_mi]

print(f"Признаки, выбранные GenericUnivariateSelect + mutual_info_classif (top {percentile}%):")
for name, score in zip(selected_features_mi, scores_mi):
    print(f"{name:40s}  MI = {score:.3f}")


Признаки, выбранные GenericUnivariateSelect + mutual_info_classif (top 50%):
num__volatile acidity                     MI = 0.077
num__residual sugar                       MI = 0.059
num__chlorides                            MI = 0.060
num__total sulfur dioxide                 MI = 0.073
num__density                              MI = 0.156
num__alcohol                              MI = 0.156


In [17]:
numeric_features = X_regression.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_regression.select_dtypes(include=['object', 'category']).columns

numeric_features, categorical_features

(Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
        'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
        'pH', 'sulphates'],
       dtype='object'),
 Index(['quality', 'type'], dtype='object'))

## Оберточные методы

Иедя методов - n раз обучить модель на подвыборках, выбрать лучшие признаки по коэфицинтеам.

- RFE (Recursive Feature Elimination)
- RFECV (Recursive Feature Elimination Cross Validation)

RFE (Recursive Feature Elimination)

`Алгоритм RFE (рекурсивное исключение признаков)`:

1. Обучаем модель (у нас LinearRegression) на всех признаках.

2. Смотрим на важность/коэффициенты (coef_).

3. Удаляем один или несколько «самых слабых» признаков.

4. Повторяем, пока не останется нужное количество признаков n_features_to_select.

**Важная деталь**: Мы заранее задаём сколько признаков хотим оставить.

`RFECV (рекурсивное исключение признаков с кросс валидацией)`

То же самое, но:

1. внутрь встроена кросс-валидация;

2. перебирается разное количество признаков (1, 2, 3, …, p);

3. выбирается то число признаков, при котором качество по CV лучше всего.

### Препроцессинг данных

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# целевая и признаки
y_regression = df['alcohol']
X_regression = df.drop(columns=['alcohol'])

numeric_features = X_regression.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_regression.select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_reg = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

prep_pipe_reg = Pipeline(steps=[
    ('preprocess', preprocessor_reg)
])

# Преобразуем X в числовую матрицу
X_reg_prepared = prep_pipe_reg.fit_transform(X_regression)
feature_names_reg = prep_pipe_reg.named_steps['preprocess'].get_feature_names_out()


### RFE + LinearRegression (фиксированное число признаков)

In [61]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

k = 8  # сколько признаков хотим оставить

base_estimator = LinearRegression()

rfe = RFE(
    estimator=base_estimator,
    n_features_to_select=k
)

X_rfe_selected = rfe.fit_transform(X_reg_prepared, y_regression)

mask_rfe = rfe.get_support()
selected_features_rfe = feature_names_reg[mask_rfe]

print(f"Признаки, выбранные RFE (k={k}):\n")
for name in selected_features_rfe:
    print(name)

# Можно посмотреть порядок исключения признаков
print("\nРанги (чем меньше, тем важнее):\n")
for name, rank in zip(feature_names_reg, rfe.ranking_):
    print(f"{name:40s}  rank = {rank}")


Признаки, выбранные RFE (k=8):

num__fixed acidity
num__residual sugar
num__density
num__pH
cat__quality_5
cat__quality_9
cat__type_red
cat__type_white

Ранги (чем меньше, тем важнее):

num__fixed acidity                        rank = 1
num__volatile acidity                     rank = 6
num__citric acid                          rank = 8
num__residual sugar                       rank = 1
num__chlorides                            rank = 11
num__free sulfur dioxide                  rank = 9
num__total sulfur dioxide                 rank = 12
num__density                              rank = 1
num__pH                                   rank = 1
num__sulphates                            rank = 2
cat__quality_3                            rank = 4
cat__quality_4                            rank = 5
cat__quality_5                            rank = 1
cat__quality_6                            rank = 10
cat__quality_7                            rank = 7
cat__quality_8                            rank

### RFECV + LinearRegression (автоматический выбор числа признаков)

In [74]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from pprint import pprint

base_estimator = LinearRegression()

cv = KFold(n_splits=5, shuffle=True, random_state=42)

rfecv = RFECV(
    estimator=base_estimator,
    step=1,            # сколько признаков удалять за раз
    cv=cv,
    scoring='neg_root_mean_squared_error',  # метрика для регрессии (scoring - параметр который максимизирует метрику качества, поэтому мы берем net_*)
    verbose=0,
)

X_rfecv_selected = rfecv.fit_transform(X_reg_prepared, y_regression)

mask_rfecv = rfecv.get_support()
selected_features_rfecv = feature_names_reg[mask_rfecv]

print("Лучшее число признаков по RFECV:", rfecv.n_features_)
print("Признаки, выбранные RFECV:")
for name in selected_features_rfecv:
    print(name)

# Можем посмотреть, как менялось качество при разных числах признаков
print("Оценки качества по количеству признаков:")
pprint(rfecv.__dict__)


Лучшее число признаков по RFECV: 18
Признаки, выбранные RFECV:
num__fixed acidity
num__volatile acidity
num__citric acid
num__residual sugar
num__chlorides
num__free sulfur dioxide
num__density
num__pH
num__sulphates
cat__quality_3
cat__quality_4
cat__quality_5
cat__quality_6
cat__quality_7
cat__quality_8
cat__quality_9
cat__type_red
cat__type_white
Оценки качества по количеству признаков:
{'cv': KFold(n_splits=5, random_state=42, shuffle=True),
 'cv_results_': {'mean_test_score': array([-0.86733941, -0.81177356, -0.81177356, -0.72100646, -0.63259489,
       -0.52794619, -0.52433305, -0.5143761 , -0.51295978, -0.50835094,
       -0.50549388, -0.5025688 , -0.50177461, -0.5005381 , -0.49997186,
       -0.49881091, -0.49893777, -0.49831353, -0.49857652]),
                 'n_features': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                 'split0_test_score': array([-0.85696011, -0.80168567, -0.80168567, -0.71254447, -0.60826017,
    

## Часть 2 - Трансформеры, Pipeline

### Трансормеры (sklearn)

В sklearn (а впоследствии и в остальных либах) вся идея крутится вокруг трех базовых вещей

- Estimator (оцениватель) - что-то, что умеет делать `.fit()` # обучиться на данных
- Transformer (преобразователь) - что-то, что умеет `.fit()`, `.transform()` # обучиться и преобразовать
- Model/Predictor - что-то, что умеет `.fit()` и `.predict()` # обучиться и предсказать

```python
transformer.fit(X, y=None)       # "учится", как преобразовывать
X_new = transformer.transform(X) # применяет преобразование
```

Примеры стандартных трансформеров:
- StandardScaler, MinMaxScaler
- OneHotEncoder
- SelectKBest
- ColumnTransformer (трансформер над столбцами)

#### ColumnTransformer

`ColumnTransformer` — это трансформер, который:

- позволяет разные преобразования для разных групп столбцов
(числовые отдельно, категориальные отдельно, и т.д.);

- на вход принимает сырые DataFrame/массив с колонками;

- на выход даёт одну матрицу признаков, в которой всё уже числовое.

`ColumnTransformer` сам по себе — трансформер, с ним можно делать `fit`, `transform`, `fit_transform`, включать его в `Pipeline`.

``` python
ColumnTransformer(
    transformers,
    remainder='drop',
    sparse_threshold=0.3,
    n_jobs=None,
    verbose=False
)
```

`transformers` — список «блоков», каждый блок это кортеж:

```python
('имя_блока', transformer, список_колонок)

('standard_scaling', StandardScaler(), columns -> список колонок, можно не указывать, тогда применяем ко всему)
```

```python
transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
]
```

`remainder` — что делать с колонками, которые не попали ни в один блок:
- `drop` - удаляем (по умолчанию)
- `passthrough` - игнорировать

`sparse_threshold` - после какой доли преобразовывать в numpy массив. Обычно не используем, но это доля ненулевых значений в %, после которого мы преобразуем в numpy матрицу

In [78]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = df.drop(columns=['quality'])
y = df['quality']

numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'   # остальные колонки выкидываем (у нас их нет)
)

X_prepared = preprocessor.fit_transform(X)
feature_names = preprocessor.get_feature_names_out()


### Pipeline - конвеер

Pipeline — это «конвейер» из шагов:

- каждый шаг — трансформер;

- последний шаг — либо трансформер, либо модель (с predict) (если последний шаг пайплайна - модель).

Идея пайплайна - собрать все этапы обработки в один конвеер, то есть последовательность вызовов у нас всгеда такая

```python
X1 = step1.fit_transform(X)
X2 = step2.fit_transform(X1)
...
y_pred = last_step.predict(X_last)
```

Конструктор пайплайна

```python
Pipeline(
    steps,
    memory=None,
    verbose=False
)
```

`steps` - список шагов

```python
steps = [
    ('preprocess', preprocessor),
    ('select', selector),
    ('model', classifier)
]
```

`memory` - мемоизация результата (сохраниение в память)

У пайплайна есть методы:

- fit(X, y) — обучает все шаги по очереди;

- transform(X) — если последний шаг — трансформер;

- predict(X) — если последний шаг умеет predict;

- score(X, y) — вызывает score последнего шага.

In [79]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

X = df.drop(columns=['quality'])
y = df['quality']

numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['category']).columns

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

clf = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', KNeighborsClassifier(n_neighbors=5))
])

scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
print("Средняя accuracy по CV:", scores.mean())


Средняя accuracy по CV: 0.44191093740747317


#### Добавим промужеточный шаг - фильтр

In [80]:
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import Pipeline

clf_fs = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('select', SelectPercentile(score_func=f_classif, percentile=30)),
    ('model', KNeighborsClassifier(n_neighbors=5))
])

scores_fs = cross_val_score(clf_fs, X, y, cv=5, scoring='accuracy')
print("Средняя accuracy с отбором признаков:", scores_fs.mean())


Средняя accuracy с отбором признаков: 0.42635743471309295


#### Попробуем коннвер обучения и тестирования на разных моделях

In [81]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile, f_classif

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


X = df.drop(columns=['quality'])
y = df['quality']

numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns


In [84]:
numeric_transformer_linear = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer_ohe = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_linear = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_linear, numeric_features),
        ('cat', categorical_transformer_ohe, categorical_features)
    ]
)


In [85]:
preprocessor_tree = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer_ohe, categorical_features)
    ],
    remainder='passthrough'   # для DecisionTree не нужно преобразовывать признаки
)


In [86]:
def make_pipeline(model, use_tree_style=False, percentile=30):
    """
    model          — любой классификатор (LogReg, KNN, DecisionTree, ...).
    use_tree_style — если True, берём препроцессинг без скейлинга (для дерева).
    percentile     — какой процент признаков оставить.
    """
    steps = []

    if use_tree_style:
        steps.append(('preprocess', preprocessor_tree))
    else:
        steps.append(('preprocess', preprocessor_linear))

    steps.append(('select', SelectPercentile(score_func=f_classif, percentile=percentile)))
    steps.append(('model', model))

    return Pipeline(steps)


In [110]:
logreg_clf = make_pipeline(
    model=LogisticRegression(max_iter=1000),
    use_tree_style=False,      # нужен скейлинг
    percentile=30
)


In [89]:
knn_clf = make_pipeline(
    model=KNeighborsClassifier(n_neighbors=11),
    use_tree_style=False,      # тоже лучше со скейлингом
    percentile=30
)


In [93]:
tree_clf = make_pipeline(
    model=DecisionTreeClassifier(
        max_depth=5,
        random_state=42
    ),
    use_tree_style=True,       # отдельный препроцессор без StandardScaler
    percentile=60
)


In [111]:
models = {
    'LogisticRegression': logreg_clf,
    'KNN': knn_clf,
    'DecisionTree': tree_clf
}

for name, clf in models.items():
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print(f"{name:16s}  accuracy = {scores.mean():.3f} ± {scores.std():.3f}")


LogisticRegression  accuracy = 0.509 ± 0.028
KNN               accuracy = 0.460 ± 0.024
DecisionTree      accuracy = 0.478 ± 0.057


## Часть 3 - Кросс-валидация

`Кросс-валидация` ― это способ оценить качество модели, не переобучив её на тесте.

идея:

1. Делим данные на несколько частей (фолдов).

2. Пока одна часть используется как валидация, остальные ― обучение.

3. Повторяем так, чтобы каждая часть стала валидацией.

4. Усредняем качество по всем «прогонам».

Зачем это нужно:

- мы получаем стабильную, менее шумную оценку;

- используем все данные и для обучения, и для проверки;

- нет риска, что случайное разбиение даст «слишком удачные» или «слишком плохие» результаты.

`K-Fold Cross-Validation`


1. Делим данные на k равных блоков.

2. Берём 1 блок как validation, остальные k−1 ― train.

3. Повторяем k раз.

Итоговая метрика = среднее по k оценкам.

![img](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [109]:
from sklearn.model_selection import KFold, cross_val_score

logreg_clf = make_pipeline(
    model=LogisticRegression(max_iter=1000),
    use_tree_style=False,      # нужен скейлинг
    percentile=30
)

scores = cross_val_score(logreg_clf, X, y, cv=4, scoring='accuracy')
print("K-Fold scores:", scores)
print(f"Средняя точность: {scores.mean():.3f}", f" Средний разброс: +-{scores.std():.3f}", )


K-Fold scores: [0.46523077 0.51724138 0.54125616 0.50800493]
Средняя точность: 0.508  Средний разброс: +-0.027


`ShuffleSplit` - многократные случайные разбиения.

1. Каждый раз случайно выбирается часть train и часть validation.

2. Количество повторов (n_splits) можно задавать любое.

![1](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_008.png)

In [112]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(
    n_splits=5,
    test_size=0.2,
    random_state=42
)

logreg_clf = make_pipeline(
    model=LogisticRegression(max_iter=1000),
    use_tree_style=False,
    percentile=30
)

scores = cross_val_score(logreg_clf, X, y, cv=ss, scoring='accuracy')
print("ShuffleSplit:", scores, "Средняя точность:", scores.mean())


ShuffleSplit: [0.52615385 0.54538462 0.52615385 0.52461538 0.54615385] Средняя точность: 0.5336923076923077
